In [1]:
# Import dependencies
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import requests
import pymongo
import os
import time
from config import nasa_url, facts_url, mars_hemisphere_url

In [3]:
# URL of page to be scraped
url = nasa_url

# Retrieve page with the requests module
response = requests.get(url, verify=True)

# Parse HTMl
soup = bs(response.text, 'html.parser')

In [4]:
# Grab latest headline and tagline
headline = soup.find_all('div', class_="content_title")[0].text
print(headline)

# Grab the tagline for the above headline
tagline = soup.find_all('div', class_="rollover_description_inner")[0].text
print(tagline)



NASA's Perseverance Rover Collects Puzzle Pieces of Mars' History



The rocks it has analyzed for sample collection are helping the team better understand a past marked by volcanic activity and water.



In [5]:
# Use Splinter to visit site for featured image - First init Splinter
executable_path = {'executable_path': ChromeDriverManager().install()} # Ensure latest version is installed
browser = Browser('chrome', **executable_path, headless=False) # Initialize browser

# Visit URL
ft_img_grab_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(ft_img_grab_url)

# Open featured image full res destination
browser.links.find_by_partial_text('FULL IMAGE').click()
time.sleep(6)

# Parse scraped HTML
scraped_html = browser.html
img_parse = bs(scraped_html, 'html.parser')

# Grab full res image
featured_image_scrape = img_parse.find('img', class_="fancybox-image")
featured_image_url = featured_image_scrape.attrs['src']

# Get full link
featured_img_final = f"https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{featured_image_url}"
featured_img_final



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/josephchancey/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

In [6]:
# Closer Splinter browser
browser.quit()

In [7]:
# Scrape table with Pandas
table_url = "https://space-facts.com/mars/"

# read_html to automatically scrape every table from page
tables = pd.read_html(requests.get(table_url).text)
tables
# Assign to dataframe to view data - [0] index is the table we wanted
df = tables[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [8]:
# Clean DataFrame
# Rename column
df = df.rename(columns={1: "Values", 0:"Query"})
# Set New Index
df = df.set_index('Query')
df

,Values
Query,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [9]:
# Get hemisphere data 

# Scrape Titles
# Visit URL
browser.visit(mars_hemisphere_url)
hemisphere_img_dict = []
hemisphere_title_dict = []

# Parse HTML
hemi_html = browser.html
hemi_parse = bs(hemi_html, 'html.parser')

# Initial scrape to gather hemispheres 
total_hemis = soup.find_all('div', class_="item")

# Loop and gather titles 
for hemi in totla_hemis:
    hemisphere_title_dict.append(total_hemis.find('h3').text.strip())
    
hemisphere_title_dict

# for hemi in range(len(total_hemis)): # Loop through every hemisphere on page and scrape data
#     # Click on each Hemisphere URL
#     browser.links.find_by_css('itemLink').click()
#     time.sleep(2)
#     # Parse scraped HTML from each Hemisphere
#     hemi_html = browser.html
#     hemi_parse = bs(hemi_html, 'html.parser')

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=54182): Max retries exceeded with url: /session/66fd4487f226aac3a630ed646940402b/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb1837c6550>: Failed to establish a new connection: [Errno 61] Connection refused'))